In [1]:
import sys
!{sys.executable} -m pip install google-cloud-bigquery

  Obtaining dependency information for google-cloud-bigquery from https://files.pythonhosted.org/packages/51/8c/bf168c5450431734d67ed4db3e62e2c81fbf2c7d8c0ff3153808e9ab480f/google_cloud_bigquery-3.13.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0dev,>=1.31.5 from https://files.pythonhosted.org/packages/c4/1e/924dcad4725d2e697888e044edf7a433db84bf9a3e40d3efa38ba859d0ce/google_api_core-2.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0dev,>=1.15.0 from https://files.pythonhosted.org/packages/36/5b/e02636d221917d6fa2a61289b3f16002eb4c93d51c0191ac8e896d527182/proto_plus-1.22.3-py3-none-any.whl.metadata
  Using cached proto_plus-1.22.3-py3-none-any.whl.metadata (2.2 kB)
  Obtaining dependency information for google-cloud-core<3.0.0dev,>=1.6.0 from https://files.pythonhosted.org/packages/a2/40/02045f776fdb6e44194f34b6375a26ce8a61bd9bd03cd8930ed91cf51a62/google_cloud_core-


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from google.cloud import bigquery

#Initialize the BigQuery client
client = bigquery.Client(project="degroup11")

# Define the PySpark schema for the streaming data
data_schema = StructType([
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Price", IntegerType(), True),
    StructField("Lot_size", StringType(), True),
    StructField("Living_space_size", StringType(), True),
    StructField("Build_year", StringType(), True),
    StructField("Build_type", StringType(), True),
    StructField("House_type", StringType(), True),
    StructField("Roof", StringType(), True),
    StructField("Rooms", StringType(), True),
    StructField("Toilet", StringType(), True),
    StructField("Floors", StringType(), True),
    StructField("Energy_label", StringType(), True),
    StructField("Position", StringType(), True),
    StructField("Garden", StringType(), True),
    StructField("Estimated_neighbourhood_price_per", StringType(), True),
    StructField("Availability", BooleanType(), True),
    StructField("event_time",TimestampType(), True),
])


# Specify the BigQuery dataset and table
dataset_id = "group11dataset"  
table_id = "house_pricing_kafka"   

# Recreate the BigQuery table based on this https://stackoverflow.com/questions/10604135/google-bigquery-delete-rows it is more economical to delete and create a table
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=data_schema)

#Delete the BigQuery table
client.delete_table(table_id, not_found_ok=True)

# Create the table if it doesn't exist
client.create_table(table, exists_ok=True)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType
from pyspark.sql.window import Window
from time import sleep


  

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Lab9_Ex3")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# We need to set the following configuration whenever we need to use GCS.
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "temp_degroup11"
spark.conf.set('temporaryGcsBucket', bucket)

# Define the PySpark schema for the streaming data
data_schema = StructType([
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Price", IntegerType(), True),
    StructField("Lot_size", StringType(), True),
    StructField("Living_space_size", StringType(), True),
    StructField("Build_year", StringType(), True),
    StructField("Build_type", StringType(), True),
    StructField("House_type", StringType(), True),
    StructField("Roof", StringType(), True),
    StructField("Rooms", StringType(), True),
    StructField("Toilet", StringType(), True),
    StructField("Floors", StringType(), True),
    StructField("Energy_label", StringType(), True),
    StructField("Position", StringType(), True),
    StructField("Garden", StringType(), True),
    StructField("Estimated_neighbourhood_price_per", StringType(), True),
    StructField("Availability", BooleanType(), True),
    StructField("event_time",TimestampType(), True),
])




# Read the whole dataset as a batch
kafkaStream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9093") \
    .option("failOnDataLoss", "false") \
    .option("subscribe", "mock") \
    .option("startingOffsets", "latest") \
    .load()

df = kafkaStream.selectExpr("CAST(value AS STRING)")

df1 = df.select(from_json(df.value, data_schema.simpleString()))

df1.printSchema()

sdf = df1.select(col("from_json(value).*"))

sdf.printSchema()

# Filter data based on a given price X
price_threshold = 400000  # Set your price threshold

top_10_prices_df = sdf \
    .groupBy(window(col("event_time"), "10 seconds"),"Address","City", "Price","Lot_size","Living_space_size","Build_year","Build_type","House_type","Rooms","Toilet","Floors","Roof","Energy_label","Position","Garden","Estimated_neighbourhood_price_per","Availability") \
    .agg(F.max("event_time").alias("event_time")) \
    .where((col("Price") <= price_threshold) & (col("Availability") == True)) \
    .orderBy("Price", ascending=False)
 
top_10_prices_df = top_10_prices_df.dropDuplicates(["Address", "Price"])


def my_foreach_batch_function(df, batch_id):
    
    df.show()
    df.write.format('bigquery') \
      .option('table', 'degroup11.group11dataset.house_pricing_kafka') \
      .mode("append") \
      .save()

query = top_10_prices_df.writeStream.outputMode("complete") \
                    .trigger(processingTime = '10 seconds').foreachBatch(my_foreach_batch_function).start()

try:
    query.awaitTermination()
except KeyboardInterrupt:
    query.stop()
    # Stop the spark context
    spark.stop()
    print("Stopped the streaming query and the spark context")

In [ ]:
spark.stop()